In [8]:

import requests
import pandas as pd

url = 'https://pokeapi.co/api/v2/pokemon?limit=50'

def main_request(url):
    r = requests.get(url)
    return r.json()

def parse_data(resp):
    all_pokemon = []
    results = resp.get('results', [])

    for poke in results:
        detail_url = poke.get('url')
        detail = requests.get(detail_url).json()

        name = detail.get('name')
        types = detail.get('types', [])
        abilities = detail.get('abilities', [])
        moves = detail.get('moves', [])
        height = detail.get('height')
        weight = detail.get('weight')

        primary_type = types[0]['type']['name'] if types else None
        primary_ability = abilities[0]['ability']['name'] if abilities else None
        moves_count = len(moves)

        row = {
            'pokemon': name,
            'primary_type': primary_type,
            'primary_ability': primary_ability,
            'moves_count': moves_count,
            'height': height,
            'weight': weight
        }
        all_pokemon.append(row)

    return all_pokemon

data = main_request(url)
result = parse_data(data)

df = pd.DataFrame(result)

by_type = (
    df.groupby('primary_type')
      .size()
      .reset_index(name='count')
      .sort_values('count', ascending=False)
)
print("\nCounts by primary_type:")
print(by_type)

by_type_avg = (
    df.groupby('primary_type')
      .agg(avg_height=('height','mean'),
           avg_weight=('weight','mean'))
      .sort_values('avg_height', ascending=False)
      .reset_index()
)
print("\nAverage height and weight by primary_type:")
print(by_type_avg)

top_moves = (
    df[['pokemon','moves_count']]
      .sort_values('moves_count', ascending=False)
      .head(10)
      .reset_index()
)
print("\nTop 10 pokemon by moves_count:")
print(top_moves)


Counts by primary_type:
  primary_type  count
0          bug     10
7       poison     10
6       normal      9
4        grass      6
3         fire      5
5       ground      3
8        water      3
1     electric      2
2        fairy      2

Average height and weight by primary_type:
  primary_type  avg_height  avg_weight
0       poison   13.200000  311.900000
1        water   10.333333  390.000000
2        grass   10.333333  254.166667
3         fire   10.200000  295.600000
4        fairy    9.500000  237.500000
5          bug    7.800000  164.900000
6       normal    7.666667  167.555556
7     electric    6.000000  180.000000
8       ground    6.000000  141.000000

Top 10 pokemon by moves_count:
   index     pokemon  moves_count
0     35    clefable          163
1     34    clefairy          161
2     39  wigglytuff          153
3     38  jigglypuff          148
4     33    nidoking          134
5     30   nidoqueen          133
6      5   charizard          131
7      8   blasto